# Parsing

In [ ]:
!pip install pypdf pymorphy2 selenium pymupdf
!apt-get update
!apt install chromium-chromedriver

In [ ]:
import os, json, time, hashlib, zipfile
import warnings, gc
import requests, fitz
from itertools import islice
from io import BytesIO
import urllib3, multiprocessing
from urllib.request import urlopen
from urllib.parse import urlparse
from selenium import webdriver
from concurrent.futures import ThreadPoolExecutor, as_completed
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from pypdf import PdfReader
import pandas as pd
from IPython.display import display
import numpy as np
import re
import nltk
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer

warnings.filterwarnings("ignore")
nltk.download('stopwords')
nltk.download('punkt')
urllib3.disable_warnings()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def download_pdf(url):
    response = requests.get(url)
    if response.status_code == 200:
        original_filename = os.path.basename(url)
        truncated_filename = truncate_filename(original_filename)
        file_path = os.path.join('/content/PDFs/', truncated_filename)
        with open(file_path, 'wb') as file:
            file.write(response.content)
        return file_path
    else:
        print(f"Failed to download PDF: {url, response.status_code}")
        return None

In [ ]:
def truncate_filename(filename, max_length=255):
    if len(filename) > max_length:
        hash_part = hashlib.md5(filename.encode()).hexdigest()
        extension = os.path.splitext(filename)[1]
        filename = filename[:max_length - len(hash_part) - len(extension) - 1] + '-' + hash_part + extension
    return filename

In [ ]:
def clear_string(url):
    parts = url.split(' ')
    result = []
    i = 0
    if i + 2 < len(parts) and parts[i+1] == 'и':
        result.append(parts[i])
        result.append(parts[i+2])
    else:
        result.append(parts[i])
    return result

In [ ]:
def get_all_links(company, url):
    if ".pdf" in url.lower():
        internalLinks = []
        if "http" in url:
            filename = download_pdf(url)
            if filename is not None:
                internalLinks.append(filename)

                data = pd.DataFrame(internalLinks, columns=['urls']).drop_duplicates()
                data['company'] = company
                data = data[['company', 'urls']]

                return data

        else:
            if "/content/PDFs/" not in url:
                url = os.path.join('/content/PDFs/', url)

            internalLinks.append(url)

            data = pd.DataFrame(internalLinks, columns=['urls']).drop_duplicates()
            data['company'] = company
            data = data[['company', 'urls']]

            return data
    else:
        internalLinks = [url]
        status = 0
        try:
            response = requests.get(url=url, timeout=timeout, headers=headers, verify=False)
            status = response.status_code
        except Exception as err:
            pass

        if status == 200:
            soup = BeautifulSoup(response.content, 'lxml')
        else:
            chrome_options = webdriver.ChromeOptions()
            chrome_options.add_argument('--headless')
            chrome_options.add_argument('--no-sandbox')
            chrome_options.add_argument('--disable-gpu')
            chrome_options.add_argument('--disable-extensions')
            chrome_options.add_argument('--disable-dev-shm-usage')
            chrome_options.add_argument('--disable-blink-features=AutomationControlled')
            driver = webdriver.Chrome(options=chrome_options)

            driver.set_page_load_timeout(10)

            driver.get(url)
            soup = BeautifulSoup(driver.page_source, 'lxml')
            driver.quit()
            del driver
            gc.collect()

        for script in soup(["script", "style"]):
            script.extract()

        sublinks = [a.get('href') for a in soup.find_all('a') if a.get('href') and a.get('href').startswith('/')]
        sublinks = [link[1:] for link in sublinks]
        sublinks = [*set(sublinks)]
        sublinks = list(filter(None, sublinks))

        for link in sublinks:
            if ".pdf" not in link.lower():
                if 'www' in link:
                    new_url = os.path.join(url[:url.rfind('www')], urlparse(url).netloc, link)
                else:
                    new_url = os.path.join(url[:url.rfind('//')+2], urlparse(url).netloc, link)
                internalLinks.append(new_url)

        data = pd.DataFrame(internalLinks, columns=['urls']).drop_duplicates()
        data['company'] = company
        data = data[['company', 'urls']]

        return data

In [ ]:
def parsing(company, url):
    if ".pdf" in url.lower():
        if "http" in url:
            filename = download_pdf(url)
            if filename is not None:
                reader = fitz.open(f"{filename}")
                text = ""
                for page in range(reader.page_count):
                    text += reader.load_page(page).get_text() + "\n"

                text = text.replace('-\n', '').lower()
                text = re.sub(r'[`!@#$%^&*()_+\-=\[\]{};\':"\\|,.<>\/?~©«»—]', r'', text)
                text = ''.join([i for i in text if not i.isdigit()])

                lines = (line.strip() for line in text.splitlines())
                chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
                text = '\n'.join(chunk for chunk in chunks if chunk)

                text = list(np.concatenate([sent_tokenize(i.strip()) for i in text.split('\n')]).flat)

                text_lemmatized = []
                for line in [t.split() for t in text]:
                    line_lemmatized = ' '.join([morph.normal_forms(l)[0] for l in line if l not in stopwords_ru])
                    text_lemmatized.append(line_lemmatized)

                data = pd.DataFrame(text_lemmatized, columns=['text']).drop_duplicates()
                data['company'] = company
                data['url'] = url
                data = data[['company', 'url', 'text']]

                return data
        else:
            if "/content/PDFs/" not in url:
                url = os.path.join('/content/PDFs/', url)

            reader = fitz.open(f"{url}")
            text = ""
            for page in range(reader.page_count):
                text += reader.load_page(page).get_text() + "\n"

            text = text.replace('-\n', '').lower()
            text = re.sub(r'[`!@#$%^&*()_+\-=\[\]{};\':"\\|,.<>\/?~©«»—]', r'', text)
            text = ''.join([i for i in text if not i.isdigit()])

            lines = (line.strip() for line in text.splitlines())
            chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
            text = '\n'.join(chunk for chunk in chunks if chunk)

            text = list(np.concatenate([sent_tokenize(i.strip()) for i in text.split('\n')]).flat)

            text_lemmatized = []
            for line in [t.split() for t in text]:
                line_lemmatized = ' '.join([morph.normal_forms(l)[0] for l in line if l not in stopwords_ru])
                text_lemmatized.append(line_lemmatized)

            data = pd.DataFrame(text_lemmatized, columns=['text']).drop_duplicates()
            data['company'] = company
            data['url'] = url
            data = data[['company', 'url', 'text']]

            return data

    else:
        status = 0
        try:
            response = requests.get(url=url, timeout=timeout, headers=headers, verify=False)
            status = response.status_code
        except Exception as err:
            pass

        if status == 200:
            soup = BeautifulSoup(response.content, 'lxml')
        else:
            chrome_options = webdriver.ChromeOptions()
            chrome_options.add_argument('--headless')
            chrome_options.add_argument('--no-sandbox')
            chrome_options.add_argument('--disable-gpu')
            chrome_options.add_argument('--disable-extensions')
            chrome_options.add_argument('--disable-dev-shm-usage')
            chrome_options.add_argument('--disable-blink-features=AutomationControlled')
            driver = webdriver.Chrome(options=chrome_options)

            driver.set_page_load_timeout(10)
            driver.get(url)
            soup = BeautifulSoup(driver.page_source, 'lxml')
            driver.quit()
            del driver
            gc.collect()

        for script in soup(["script", "style"]):
            script.extract()

        text = soup.get_text()

        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = '\n'.join(chunk for chunk in chunks if chunk).replace('-\n', '').lower()
        text = re.sub(r'[`!@#$%^&*()_+\-=\[\]{};\':"\\|,.<>\/?~©«»—]', r'', text)
        text = ''.join([i for i in text if not i.isdigit()])

        text = list(np.concatenate([sent_tokenize(i.strip()) for i in text.split('\n')]).flat)

        text_lemmatized = []
        for line in [t.split() for t in text]:
            line_lemmatized = ' '.join([morph.normal_forms(l)[0] for l in line if l not in stopwords_ru])
            text_lemmatized.append(line_lemmatized)

        spans = list(filter(None, [span.string for span in soup.find_all('span')]))

        if len(spans):
            spans = '\n'.join(span for span in spans if span).replace('-\n', '').lower()
            spans = re.sub(r'[`!@#$%^&*()_+\-=\[\]{};\':"\\|,.<>\/?~©«»—]', r'', spans)
            spans = ''.join([i for i in spans if not i.isdigit()])

            spans = list(np.concatenate([sent_tokenize(i.strip()) for i in spans.split('\n')]).flat)
        else:
            spans = []

        spans_lemmatized = []
        for line in [s.split() for s in spans]:
            line_lemmatized = ' '.join([morph.normal_forms(l)[0] for l in line if l not in stopwords_ru])
            spans_lemmatized.append(line_lemmatized)

        df1 = pd.DataFrame(text_lemmatized, columns=['text'])
        df2 = pd.DataFrame(spans_lemmatized, columns=['text'])
        data = pd.concat([df1, df2]).drop_duplicates().reset_index(drop=True)
        data['company'] = company
        data['url'] = url
        data = data[['company', 'url', 'text']]

        return data

In [ ]:
def chunkify(data, chunk_size):
    it = iter(data)
    for i in range(0, len(data), chunk_size):
        yield {k: data[k] for k in [next(it) for _ in range(min(chunk_size, len(data) - i))]}

def process_chunk(URLs_chunk, enum, function):
    with ThreadPoolExecutor(4) as executor:
        chunk_res = pd.DataFrame()
        results = {executor.submit(function, company, url): (url, company) for url, company in URLs_chunk.items()}
        for i, result in enumerate(as_completed(results)):
            try:
                url, company = results[result]
                df1 = result.result()
                chunk_res = pd.concat([chunk_res, df1])
                print('DONE', i+enum, company)
            except Exception as err:
                print('ERROR', i+enum, company, err)
                continue
    return chunk_res, i+enum+1

In [ ]:
def zip_folder(folder_path, output_path):
    with zipfile.ZipFile(output_path, 'w') as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), folder_path))

In [ ]:
stopwords_ru = stopwords.words("russian")
morph = MorphAnalyzer()

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    "accept": "application/xml"
}

timeout = (5, 25)

In [ ]:
raex_list = pd.read_excel("RAEX2024.xlsx")

url = 'https://docs.google.com/spreadsheets/d/1uUgKmlYBCWxFTMvbcwKQDCblTQhKOsZ_zPs0VCRQBEE/export?format=xlsx'
response = requests.get(url)

excel = pd.ExcelFile(BytesIO(response.content))
sheets = excel.sheet_names
sheets

['Методика',
 'Компании СЗ+RAEX',
 'Copy of Компании СЗ',
 'Общий отраслевой рейтинг',
 'Региональный отраслевой рейтинг',
 'Общий глобальный рейтинг ',
 'Региональный глобальный рейтинг',
 'База данных',
 'Copy of Ранги и места',
 'Компании RAEX',
 'MOEX',
 'Технический лист',
 'Sheet13',
 'Sheet14',
 'Sheet15']

In [ ]:
data = pd.read_excel(excel, sheet_name='Компании СЗ+RAEX')
data.drop(['Unnamed: 0', 'Unnamed: 17', 'Unnamed: 18'], axis=1, inplace=True)
data.columns = ['Отрасль', 'Бренд и Юрлицо', 'Веб-сайт', 'ИНН', 'МOEX', 'Регион', 'Устойчивое развитие, Социальная отчетность, ESG', 'Экология', 'Забота о потребителях, защита прав потребителей',
                'Забота о работниках, социальные программы', 'Поставщикам, Закупки', 'Корпоративное управление, совет директоров, высший менеджмент',
                'Миссия, ценности', 'Этический кодекс', 'Антикоррупция', 'Раскрытие информации']
data = data.drop([0, 1], axis=0).reset_index(drop=True)
data['Отрасль'] = data['Отрасль'].fillna(method='ffill')
#data = data[data['Регион'] == 'RAEX']
data = data.rename(columns={'Бренд и Юрлицо': 'Название'})
data = data.drop_duplicates(subset=['Название']).reset_index(drop=True)
data.loc[data['Название'] == 'СБЕРБАНК', 'Название'] = 'Сбербанк'
data.loc[data['Название'] == 'Х5 (Вятерочка, Перекресток)', 'Название'] = 'X5 Group'
data.loc[data['Название'] == 'МТС', 'Название'] = 'ПАО "МТС"'
data.loc[data['Название'] == 'Билайн (Вымпелком СЗФО)', 'Название'] = '«Вымпелком»'
data.loc[data['Название'] == 'Segezha Group', 'Название'] = 'Сегежа Групп'
data.loc[data['Название'] == 'ГК “Самолет”', 'Название'] = 'ГК «САМОЛЕТ»'
data.loc[data['Название'] == 'АО "Группа "Илим"', 'Название'] = '«Илим», группа'
data.loc[data['Название'] == 'Группа ЛСР', 'Название'] = '«Группа ЛСР»'
data.loc[data['Название'] == 'ЛЕНТА', 'Название'] = '«Лента» (сеть гипермаркетов)'
data.loc[data['Название'] == 'Ростелеком', 'Название'] = '«Ростелеком»'
data.loc[data['Название'] == 'МАГНИТ', 'Название'] = '«Магнит», розничная сеть'
data.loc[data['Название'] == 'ВТБ', 'Название'] = 'Банк ВТБ'
data.loc[data['Название'] == 'СВЕЗА', 'Название'] = 'Свеза'
data.loc[data['Название'] == 'Мегафон', 'Название'] = '«МегаФон»'
data.loc[data['Название'] == '"Альфа-Банк"', 'Название'] = 'Альфа-Банк'
data.loc[data['Название'] == '"Открытие"', 'Название'] = '«ФК Открытие»'
data.loc[data['Название'] == 'ПСБ по СЗФО', 'Название'] = 'Промсвязьбанк'
data.loc[data['Название'] == '"ВкусВилл"', 'Название'] = '«Вкусвилл»'
data.loc[data['Название'] == '«Карелия Палп»', 'Название'] = 'Карелия Палп'
data.loc[data['Название'] == 'Азбука вкуса', 'Название'] = '«Азбука Вкуса» (сеть супермаркетов)'

data = raex_list.merge(data, on='Название', how='left')
data.drop(['url', 'url_sustainability'], axis=1, inplace=True)
data

,№,Название,Код MOEX,Подотрасль,ESG-рейтинг,E Rank,E-рейтинг,S Rank,S-рейтинг,G Rank,...,"Устойчивое развитие, Социальная отчетность, ESG",Экология,"Забота о потребителях, защита прав потребителей","Забота о работниках, социальные программы","Поставщикам, Закупки","Корпоративное управление, совет директоров, высший менеджмент","Миссия, ценности",Этический кодекс,Антикоррупция,Раскрытие информации
0,1,Сбербанк,SBER,Банки,AA,2,AA,1,AAA,4,...,https://www.sberbank.com/ru/sustainability,https://www.sberbank.com/ru/sustainability,https://www.sberbank.com/common/img/uploaded/r...,https://rabota.sber.ru,Нет,https://www.sberbank.com/ru/investor-relations...,https://www.sberbank.com/ru/about/mission,https://www.sberbank.com/ru/about/ethics,https://www.sberbank.com/ru/compliance/ukipk,https://www.sberbank.com/ru/investor-relations...
1,2,«ФосАгро»,PHOR,Агрохимикаты,AA,5,AA,3,AA,2,...,https://www.phosagro.ru/sustainability/,https://www.phosagro.ru/sustainability/ecology/,https://www.phosagro.ru/production/attention/,https://www.phosagro.ru/sustainability/social-...,https://www.phosagro.ru/procurement/,https://www.phosagro.ru/about/management/,https://www.phosagro.ru/sustainability/,https://ar2022.phosagro.ru/pdf/ar/ru/corporate...,нет,https://www.phosagro.ru/ori/phosagro/company_c...
2,3,«Полюс»,PLZL,Драгоценные металлы,AA,1,AA,2,AA,14,...,https://sustainability.polyus.com/ru/,https://sustainability.polyus.com/ru/environment/,https://sustainability.polyus.com/ru/human_rig...,https://sustainability.polyus.com/ru/our_people/,нет,https://polyus.com/ru/company/corporate_govern...,нет,нет,https://polyus.com/ru/anti-corruption/,нет
3,4,"«Татнефть», группа",TATN,Интегрированные нефтегазовые компании,AA,13,A,5,AA,6,...,https://www.tatneft.ru/sustainable_development,https://www.tatneft.ru/ekologiya,https://www.tatneft.ru/sustainable_development...,https://www.tatneft.ru/sustainable_development...,Нет,https://www.tatneft.ru/o-kompanii/corporate_go...,https://www.tatneft.ru/o-kompanii/mission,нет,https://www.tatneft.ru/o-kompanii/corporate_go...,https://www.tatneft.ru/aktsioneram-i-investora...
4,5,"«Роснефть», нефтяная компания",ROSN,Интегрированные нефтегазовые компании,AA,4,AA,22,A,5,...,Rosneft_CSR2022_RUS.pdf,Нет,Нет,https://www.rosneft.ru/Development/personnel/ ...,Нет,https://www.rosneft.ru/governance/,https://www.rosneft.ru/about/strategy/,https://www.rosneft.ru/Investors/corpgov/,https://www.rosneft.ru/Investors/corpgov/,https://www.rosneft.ru/Investors/information/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,156,Новикомбанк,-,Банки,C,114,C,145,C,155-156,...,https://ar2021.novikom.ru/ru/sustainable-devel...,Нет,Нет,Нет,https://novikom.ru/tenders/,https://novikom.ru/about/ustav-i-vnutrennie-do...,нет,нет,https://novikom.ru/anti-corruption/,https://novikom.ru/information-disclosure/
156,157-160,«Квадра»,TGKD,Электроэнергетика,M,157-160,M,157-160,M,157-160,...,https://www.quadra.ru/o-kompanii/sotsialnoe-ra...,https://www.quadra.ru/o-kompanii/sotsialnoe-ra...,нет,https://www.quadra.ru/o-kompanii/sotsialnoe-ra...,https://www.quadra.ru/trade/legal/,https://www.quadra.ru/aktsioneram-i-investoram...,нет,нет,https://www.quadra.ru/o-kompanii/protivodejstv...,https://www.quadra.ru/aktsioneram-i-investoram...
157,157-160,«Русская медная компания»,-,Производство меди,M,157-160,M,157-160,M,157-160,...,https://rmk-group.ru/ru/sustainable_developmen...,сайт не работает,сайт не работает,сайт не работает,сайт не работает,сайт не работает,сайт не работает,сайт не работает,сайт не работает,сайт не работает
158,157-160,«УГМК»,-,Добыча прочих полезных ископаемых,M,157-160,M,157-160,M,157-160,...,https://ugmk.com/about_company/strategy/,сайт не работает,сайт не работает,сайт не работает,сайт не работает,сайт не работает,сайт не работает,сайт не работает,сайт не работает,сайт не работает


In [ ]:
df = data.melt(id_vars=['№', 'Название'], value_vars=['Устойчивое развитие, Социальная отчетность, ESG', 'Экология', 'Забота о потребителях, защита прав потребителей',
                'Забота о работниках, социальные программы', 'Поставщикам, Закупки', 'Корпоративное управление, совет директоров, высший менеджмент',
                'Миссия, ценности', 'Этический кодекс', 'Антикоррупция', 'Раскрытие информации'], var_name='url_type', value_name='url')
df = df.drop(columns=['url_type'])
df

,№,Название,url
0,1,Сбербанк,https://www.sberbank.com/ru/sustainability
1,2,«ФосАгро»,https://www.phosagro.ru/sustainability/
2,3,«Полюс»,https://sustainability.polyus.com/ru/
3,4,"«Татнефть», группа",https://www.tatneft.ru/sustainable_development
4,5,"«Роснефть», нефтяная компания",Rosneft_CSR2022_RUS.pdf
...,...,...,...
1595,156,Новикомбанк,https://novikom.ru/information-disclosure/
1596,157-160,«Квадра»,https://www.quadra.ru/aktsioneram-i-investoram...
1597,157-160,«Русская медная компания»,сайт не работает
1598,157-160,«УГМК»,сайт не работает


In [ ]:
words_to_filter = ['Нет', 'нет', 'не работает']
pattern = '|'.join(words_to_filter)

df['url'] = df['url'].apply(lambda x: np.nan if pd.notna(x) and pd.Series(x).str.contains(pattern).any() else x)
df = df.dropna(subset=["url"])
df['url'] = df['url'].apply(clear_string)
df = df.explode('url')
df = df.sort_values(by='Название').reset_index(drop=True)
df

,№,Название,url
0,60,"""Каустик"" (г. Волгоград)",https://www.kaustik.ru/ru/index.php/partneram/...
1,60,"""Каустик"" (г. Волгоград)",sustainability-report-kaustik.pdf
2,60,"""Каустик"" (г. Волгоград)",https://www.kaustik.ru/ru/index.php/o-kompanii...
3,60,"""Каустик"" (г. Волгоград)",https://www.kaustik.ru/ru/index.php/o-kompanii...
4,60,"""Каустик"" (г. Волгоград)",https://www.kaustik.ru/ru/index.php/o-kompanii...
...,...,...,...
892,50,Якутская топливно-энергетическая компания (ЯТЭК),https://yatec.ru/postavshchikam/tekushchie-zak...
893,50,Якутская топливно-энергетическая компания (ЯТЭК),https://www.yatec.ru/aktsioneram/raskrytie-inf...
894,50,Якутская топливно-энергетическая компания (ЯТЭК),https://yatec.ru/aktsioneram/raskrytie-informa...
895,50,Якутская топливно-энергетическая компания (ЯТЭК),https://yatec.ru/razvitie/ecology/


In [ ]:
urls = df.set_index('url')['Название'].to_dict()
len(urls)

818

In [ ]:
folder_name = 'PDFs'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

In [ ]:
%%time
chunk_size = 5
links = pd.DataFrame()
i = 0
for URLs_chunk in chunkify(urls, chunk_size):
    chunk_results, i = process_chunk(URLs_chunk, i, get_all_links)
    try:
        links = pd.concat([links, chunk_results])
        del chunk_results
        gc.collect()
    except Exception as err:
        continue

print("Links are collected!")

links = links.drop_duplicates().reset_index(drop=True)

words_to_filter = ['.xlsx', '.docx', '.7z', '.rar', '.JPG', '.xlsb', '.xls', '.zip', '.mp4', '.jpg', '.png', '.doc', '.PNG', 'download_file.php']
links = links[~links['urls'].str.contains('|'.join(words_to_filter))].reset_index(drop=True)
links.to_csv('links.csv', encoding='utf-8')

DONE 0 "Каустик" (г. Волгоград)
DONE 1 "Каустик" (г. Волгоград)
DONE 2 "Каустик" (г. Волгоград)
DONE 3 "Каустик" (г. Волгоград)
DONE 4 "Каустик" (г. Волгоград)
DONE 5 "Нефтиса", нефтяная компания
DONE 6 "Нефтиса", нефтяная компания
DONE 7 "Нефтиса", нефтяная компания
DONE 8 "Каустик" (г. Волгоград)
DONE 9 "Каустик" (г. Волгоград)
DONE 10 "Нефтиса", нефтяная компания
DONE 11 "Нефтиса", нефтяная компания
DONE 12 "Соликамскбумпром"
DONE 13 "Соликамскбумпром"
DONE 14 "Соликамскбумпром"
DONE 15 "Соликамскбумпром"
DONE 16 "Соликамскбумпром"
DONE 17 «Титан», группа компаний (нефтехимия)
DONE 18 «Титан», группа компаний (нефтехимия)
DONE 19 "Титан", группа компаний (деревообработка)
DONE 20 "Титан", группа компаний (деревообработка)
DONE 21 "Титан", группа компаний (деревообработка)
DONE 22 En+ Group
DONE 23 En+ Group
DONE 24 En+ Group
DONE 25 En+ Group
DONE 26 En+ Group
DONE 27 Global Ports
DONE 28 Global Ports
DONE 29 En+ Group
DONE 30 Global Ports
DONE 31 Global Ports
DONE 32 Global Ports
D

In [ ]:
folder_path = 'PDFs'
output_path = 'PDFs.zip'

zip_folder(folder_path, output_path)

In [ ]:
links = pd.read_csv('links.csv', index_col=0)

with zipfile.ZipFile('PDFs.zip', 'r') as zipf:
    zipf.extractall('PDFs')

print(len(links.company.unique()))

urls = links.set_index('urls')['company'].to_dict()
print(len(urls))

143
9399


In [ ]:
%%time
chunk_size = 4
parsed_data = pd.DataFrame()
i = 0
for URLs_chunk in chunkify(urls, chunk_size):
    chunk_results, i = process_chunk(URLs_chunk, i, parsing)
    try:
        parsed_data = pd.concat([parsed_data, chunk_results])
        del chunk_results
        gc.collect()
    except Exception as err:
        continue

print("Parcing is finished!")

parsed_data = parsed_data.drop_duplicates()
parsed_data = parsed_data.reset_index(drop=True)
parsed_data.to_csv('parsed_data_train.csv', encoding='utf-8')

Выходные данные были обрезаны до нескольких последних строк (5000).
#3 0x5a09817083f5 <unknown>
#4 0x5a0981706336 <unknown>
#5 0x5a09817069af <unknown>
#6 0x5a0981717287 <unknown>
#7 0x5a098172d55e <unknown>
#8 0x5a0981732c5b <unknown>
#9 0x5a09817070fa <unknown>
#10 0x5a098172d14f <unknown>
#11 0x5a09817af195 <unknown>
#12 0x5a0981790253 <unknown>
#13 0x5a09817601c7 <unknown>
#14 0x5a0981760b3e <unknown>
#15 0x5a09819fe2cb <unknown>
#16 0x5a0981a02377 <unknown>
#17 0x5a09819eadfe <unknown>
#18 0x5a0981a02e42 <unknown>
#19 0x5a09819cf79f <unknown>
#20 0x5a0981a27178 <unknown>
#21 0x5a0981a2734b <unknown>
#22 0x5a0981a36fbc <unknown>
#23 0x7bab57d41ac3 <unknown>

ERROR 7730 Московская Биржа Message: timeout: Timed out receiving message from renderer: -0.010
  (Session info: chrome-headless-shell=125.0.6422.78)
Stacktrace:
#0 0x598ee4defe8a <unknown>
#1 0x598ee4ad945c <unknown>
#2 0x598ee4ac0816 <unknown>
#3 0x598ee4ac03f5 <unknown>
#4 0x598ee4abe336 <unknown>
#5 0x598ee4abe9af <unknown>

In [ ]:
len(parsed_data.company.unique())

130

In [ ]:
parsed_data

,company,url,text
0,"""Каустик"" (г. Волгоград)",https://www.kaustik.ru/ru/index.php/partneram/...,обращать внимание мошеннический схема ао кауст...
1,"""Каустик"" (г. Волгоград)",https://www.kaustik.ru/ru/index.php/partneram/...,ао каустик г волгоград
2,"""Каустик"" (г. Волгоград)",https://www.kaustik.ru/ru/index.php/partneram/...,служба поддержка клиент тело факс
3,"""Каустик"" (г. Волгоград)",https://www.kaustik.ru/ru/index.php/partneram/...,email sbyturkaustikru контакт управление реали...
4,"""Каустик"" (г. Волгоград)",https://www.kaustik.ru/ru/index.php/partneram/...,главный
...,...,...,...
1197237,Якутская топливно-энергетическая компания (ЯТЭК),https://www.yatec.ru/press-centr/photogallery/,правовой информация
1197238,Якутская топливно-энергетическая компания (ЯТЭК),https://www.yatec.ru/press-centr/photogallery/,положение конфиденциальность
1197239,Якутская топливно-энергетическая компания (ЯТЭК),https://www.yatec.ru/press-centr/photogallery/,политика использование cookies
1197240,Якутская топливно-энергетическая компания (ЯТЭК),https://www.yatec.ru/press-centr/photogallery/,сделать студия
